In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPooling1D
from keras.preprocessing import text, sequence
#import wandb
#from wandb.keras import WandbCallback
import numpy as np
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
dataset1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/base-case/80_20_split.csv')

In [4]:
model = Sequential()
model.add(Embedding(90000, 50, input_length = 1000))
model.add(Dropout(0.5))
model.add(Conv1D(100, 3, padding = 'valid', activation = 'relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100, 4, padding = 'valid', activation = 'relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100, 5, padding = 'valid', activation = 'relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units =3,  activation='softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
def CNN():
    dataset = dataset1.sample(n = 12000)
    X = dataset[['OBS(mist-free)']]
    X = pd.DataFrame(X)
    lb_make = LabelEncoder()
    dataset['Category-encoded'] = lb_make.fit_transform(dataset['Category'])
    Y = dataset[['Category-encoded']]
    Y = pd.DataFrame(Y)

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, stratify = Y)
    tokenizer = text.Tokenizer(num_words=90000)
    tokenizer.fit_on_texts(X_train['OBS(mist-free)'].to_list())
    X_train1 = tokenizer.texts_to_sequences(X_train['OBS(mist-free)'].to_list())
    X_test1 = tokenizer.texts_to_sequences(X_test['OBS(mist-free)'].to_list())
    X_train = sequence.pad_sequences(X_train1, maxlen=1000)
    X_test = sequence.pad_sequences(X_test1, maxlen=1000)
    model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test, Y_test))
    y_pred = model.predict(X_test)
    predictions = np.argmax(y_pred, axis=-1)
    b = (classification_report(Y_test, predictions)).split()
    c = [float(b[7]), float(b[12]), float(b[17]), float(b[20])]
    return c

In [5]:
cnn = []
for i in range(100):
    result = CNN()
    cnn.append(result)
    #print(result)
f = pd.DataFrame(cnn)
f.to_csv("/content/drive/MyDrive/Colab Notebooks/base-case/12000/cnn(12000)_base-case-test_20(100,[3,4,5].csv")

Epoch 1/10
300/300 [==============================] - 20s 46ms/step - loss: 1.0052 - accuracy: 0.5290 - val_loss: 0.8460 - val_accuracy: 0.6158
Epoch 2/10
300/300 [==============================] - 14s 45ms/step - loss: 0.7731 - accuracy: 0.6581 - val_loss: 0.7437 - val_accuracy: 0.6750
Epoch 3/10
300/300 [==============================] - 13s 44ms/step - loss: 0.6595 - accuracy: 0.7072 - val_loss: 0.6899 - val_accuracy: 0.7113
Epoch 4/10
300/300 [==============================] - 13s 44ms/step - loss: 0.5589 - accuracy: 0.7782 - val_loss: 0.6630 - val_accuracy: 0.7342
Epoch 5/10
300/300 [==============================] - 13s 45ms/step - loss: 0.4992 - accuracy: 0.8068 - val_loss: 0.6516 - val_accuracy: 0.7329
Epoch 6/10
300/300 [==============================] - 13s 44ms/step - loss: 0.4474 - accuracy: 0.8398 - val_loss: 0.6629 - val_accuracy: 0.7300
Epoch 7/10
300/300 [==============================] - 13s 44ms/step - loss: 0.4142 - accuracy: 0.8485 - val_loss: 0.6989 - val_accuracy: